<a href="https://colab.research.google.com/github/witusj/obp/blob/master/outpatient_scheduling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Import packages
import numpy as np
import math
import random

## Setup problem definition

In [ ]:
class schedule:
  def __init__(self, beta: int, T: int, d: int, N: int):
    self.beta = beta
    self.T = T
    self.d = d
    self.N = N
    self.x:list[int] = np.repeat(0, self.T)
  def make_random_schedule(self, min_x: int, max_x: int, step: int=1):
    n = self.N
    for i in range(0,self.T, step):
      if n >= 0:
        r = random.randint(min_x, max_x)
        self.x[i] = min(r,n)
        n = n - r
        print(n, self.x[i])
      else:
        return


beta = 20
T = 48
d = 5
N = 10
s1 = schedule(20,48,5,10)
step = math.floor(beta/d)
s1.make_random_schedule(0,2, step)
print(s1.x)
print(f'total number of patients in schedule {sum(s1.x)}')

8 2
7 1
6 1
5 1
3 2
2 1
0 2
0 0
-1 0
[2 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 2 0 0 0 1 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
total number of patients in schedule 10


## Functions

In [ ]:
def recursive_sum(i):
  if i==1:
    return 1
  return i+recursive_sum(i-1)

recursive_sum(5)

15

In [ ]:
def factorial(i):
  if i==1:
    return 1
  return i * factorial(i-1)

factorial(5)

120

In [ ]:
def service(k, beta):
  return pow(beta,k)*math.exp(-beta) / factorial(k)

service(10,20)

0.005816306518345136

In [ ]:
def binomial_coef(n,k):
  return factorial(n)/(factorial(k)*factorial(n-k))

print(f'{binomial_coef(4,1)} {binomial_coef(4,2)} {binomial_coef(4,3)}')

4.0 6.0 4.0


In [ ]:
def iteration_small(x, results, I, n):
  x_new = x.copy()
  for m in range(1, I-1):
    for k in list(range(1, I))[::-1]:
      if x_new[k] > 0:
        x_new[k] -= 1
        x_new[((k + m - 1) % I) + 1] += 1
        results_temp = results(x_new, 0) # class for storing results?
        if results_temp['objVal'] < results['objVal']): # need to use eval method instead?
          return x_new, results_temp
        else: 
          x_new[k] += 1
          x_new[((k + m - 1) % I) + 1] -= 1 # store index as variable?
  return X, None

test
